In [3]:
import sys
import os
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [4]:
sys.path.append(os.path.abspath('../library'))
import data as d
import preprocess as p
import utils as u
import bayes as b
import pipeline
import results as r
import figures as figs

## 1. Load and Preprocess Data

1. Load data with ``mouse_ID`` and ``tau`` the time bin size of data.

2. Preprocess data, output includes:
    - ``spikeprob``
    - ``spikes``
    - ``position_mtx``
    - ``darktrials``
    - ``deltrials``
    - ``spikeprob_shuffled``
    - ``spikes_shuffled``

3. Create an instance of the ``MouseData`` class with ``mouse_ID`` and preprocessed output. Data structure will be printed.

4. Set ``tau`` and ``rewardzone`` for the instance.

In [ ]:
mouse_ID = 'C57_60_Octavius'
Octavius_200 = d.MouseData(mouse_ID, *p.preprocess_data(d.load_data(mouse_ID, tau=0.2)))
Octavius_200.tau = 0.2 # size of time bin of the data in seconds
Octavius_200.rewardzone = [46,47,48,49,50,51,52,53,54,55,56,57,58,59] # position bins

## 2. Firing Rates Spatial Tuning Curve

1. The pipeline does the following for both ``spikes`` and ``spikeprob``:
    - Masking data and ``position_mtx``
    - Get trial length inforamtion for the data
    - Gaussian smooth data
    - Position binning data and generate tuning curves / firing rates matrix.
    - Split data into light and dark trials.
    - Scale data by a coefficient (only ``spikes``/firing rates, not for ``spikeprob``).

2. Plot tuning curves and heatmap of a specific neuron

In [ ]:
pipeline.get_tuning_curves(Octavius_200)

In [ ]:
num_neurons = Octavius_200.spikes.shape[2]
n = np.random.randint(0, num_neurons)
figs.plot_single_tuning(Octavius_200, Octavius_200.fr_smoothed, 'spikes', neuron_idx=n)
figs.plot_single_heatmap(Octavius_200, Octavius_200.fr_smoothed, 'spikes', neuron_idx=n)

## 3. Bayesian Decoding

It works by first sorting the data by trial start location and separate them into 10 chunks. Each chunk with trials that have very close start locations.

Then the decoder runs through each chunk and compute for each train/test paradigm:
- ``lgtlgt``, train with firing rates in light and test on spikes in light
- ``drkdrk``, train with firing rates in dark and test on spikes in dark
- ``lgtdrk``, train with firing rates in light and test on spikes in dark
- ``drklgt``, train with firing rates in dark and test on spikes in light

Accuracy and Errors are computed and confusion matrices are generated. Plots are shown in Section 5.

In [ ]:
Octavius_200.posterior_allchunks, Octavius_200.decoded_pos_allchunks = pipeline.run_decoder_chunks(Octavius_200, smoothfactor=0.2)
Octavius_200.results_allchunks = pipeline.run_results_chunks(Octavius_200, Octavius_200.posterior_allchunks, Octavius_200.decoded_pos_allchunks, 46, 10)

## 4. Chance Estimates

The ``spikes_shuffled`` dataset contains ``spikes`` data that are shuffled for 100 times,  breaking the spatial patterns in the data.

Chance estimates are computed by running the decoder on these shuffled data.

Since it will take hours to run this pipeline, the output has already been stored in the directory ``variables/``. You can load the output with ``pickle`` and run ``pipeline.run_results_chance()``.

In [ ]:
posterior_allreps, decoded_pos_allreps = pipeline.run_decoder_chance(Octavius_200,100)

In [ ]:
with open('../variables/Octavius_200_posterior_allreps.pkl', 'wb') as f:
    pickle.dump(posterior_allreps, f)

with open('../variables/Octavius_200_decoded_pos_allreps.pkl', 'wb') as f:
    pickle.dump(decoded_pos_allreps, f)

In [10]:
with open('../variables/Octavius_200_posterior_allreps.pkl', 'rb') as f:
    posterior_allreps = pickle.load(f)

with open('../variables/Octavius_200_decoded_pos_allreps.pkl', 'rb') as f:
    decoded_pos_allreps = pickle.load(f)

In [ ]:
chance_results = pipeline.run_results_chance(Octavius_200, posterior_allreps, decoded_pos_allreps, 100)

## 5. Decoder Performance

Plot confusion matrix from the decoder.

Compare accuracy and errors between decoder and chance level.

In [ ]:
print(Octavius_200.results_allchunks['mean_accuracy'])
print(Octavius_200.results_allchunks['mean_error'])

figs.plot_confusion_mtx(Octavius_200, Octavius_200.results_allchunks['confusion_mtx']['lgtlgt'], 'lgtlgt')
figs.plot_confusion_mtx(Octavius_200, Octavius_200.results_allchunks['confusion_mtx']['drkdrk'], 'drkdrk')
figs.plot_confusion_mtx(Octavius_200, Octavius_200.results_allchunks['confusion_mtx']['lgtdrk'], 'lgtdrk')
figs.plot_confusion_mtx(Octavius_200, Octavius_200.results_allchunks['confusion_mtx']['drklgt'], 'drklgt')

In [ ]:
figs.plot_accuracy(Octavius_200, Octavius_200.results_allchunks, chance_results, 100, save=True)
figs.plot_errors(Octavius_200, Octavius_200.results_allchunks, chance_results, 100, save=True)